In [1]:
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt

class ImputMissingValues(BaseEstimator, TransformerMixin):

  drop_list = []

  interval_imput_dict = {}
  category_imput_dict = {}

  def __init__(self, missing_threshold = 0.0, interval_method = 'mean', category_method='mode'):
    self.missing_threshold = missing_threshold
    self.interval_method = interval_method
    self.category_method = category_method
  
  def fit(self, x, y=None):
    #calculate missing fractions
    missing_series = x.isnull().sum() / x.shape[0]
    self.missing_stats = pd.DataFrame(missing_series).rename(columns = {'index': 'feature', 0: 'missing_fraction'})

    #Sorting with highest on top
    self.missing_stats = self.missing_stats.sort_values('missing_fraction', ascending = False)
    self.record_missing = pd.DataFrame(missing_series[missing_series > self.missing_threshold]).reset_index().rename(columns = {'index': 'feature', 0: 'missing_fraction'})
    self.drop_list = list(self.record_missing['feature'])

    if self.interval_method == 'mean':
      X_data_train_means = x[[k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] != 'category']].mean()
      self.interval_imput_dict = X_data_train_means.to_dict()

    if self.category_method == 'mode':
      X_data_train_modes = x[[k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] == 'category']].mode()
      self.category_imput_dict = {k : v[0] for k, v in X_data_train_modes.to_dict().items()}

  def transform(self, x):
    res = x.drop([f for f in self.drop_list if f in x.columns], axis=1)
    res = res.fillna({f:v for f,v in self.interval_imput_dict.items() if f in x.columns})
    res = res.fillna({f:v for f,v in self.category_imput_dict.items() if f in x.columns})
    return res
  
  def fit_transform(self, x):
    self.fit(x)
    return self.transform(x)
  
  def plot_missing(self):
    if self.record_missing is None:
      raise NotImplementedError('Missing values have not been calculated. Run `fit` method')

    plt.style.use('seaborn-white')
    plt.figure(figsize = (7, 5))
    plt.hist(self.missing_stats['missing_fraction'], bins = np.linspace(0, 1, 11), edgecolor = 'k', linewidth = 1.5)
    plt.xticks(np.linspace(0, 1, 11))
    plt.xlabel('Missing Fraction', size = 14)
    plt.ylabel('Count of Features', size = 14)
    plt.title('Fraction of Missing Values Histogram', size = 16)

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import itertools

class DummyTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, nominal_feature_list = None, ignore_feature_list = [], reference_class_choice = 'medium'):
    self.ignore_feature_list = ignore_feature_list
    self.nominal_feature_list = nominal_feature_list
    if reference_class_choice not in ['first', 'last', 'medium']:
      raise ValueError('reference_class_choice must be \'first\', \'last\' or \'medium\'')
    self.reference_class_choice = reference_class_choice
  
  def fit(self, x, y=None):
    if self.nominal_feature_list is None:
      self.nominal_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] == 'category' or k[:5] in ('cbin_', 'ibin_')]
    self.nominal_feature_list = [d for d in self.nominal_feature_list if d not in self.ignore_feature_list]
    self.dummy_map = {column : {value : int(ix) for ix, value in enumerate(np.sort(x[column].unique()))} for column in self.nominal_feature_list}
    self.dummy_inv_map = {feature : {ix : value for value, ix in domain.items()} for feature, domain in self.dummy_map.items()}
    if self.reference_class_choice == 'first':
      self.dummy_reference_class = {feature : np.min([ix for value, ix in domain.items()]) for feature, domain in self.dummy_map.items() if len(domain) > 1}
    elif self.reference_class_choice == 'last':
      self.dummy_reference_class = {feature : np.max([ix for value, ix in domain.items()]) for feature, domain in self.dummy_map.items() if len(domain) > 1}
    elif self.reference_class_choice == 'medium':
      self.dummy_reference_class = {feature : np.sort([ix for value, ix in domain.items()])[(len(domain)-1)//2] for feature, domain in self.dummy_map.items() if len(domain) > 1}
    
    self.dummy_column_names = {column : [column + '_dummy_' + str(ix) for value, ix in self.dummy_map[column].items()] for column in self.nominal_feature_list}
    self.dummy_reference_column = {column : column + '_dummy_' + str(self.dummy_reference_class[column]) for column in self.nominal_feature_list if column in self.dummy_reference_class}
    dummy_inv_column_names = [[(dummy, column) for dummy in dummies] for (column, dummies) in self.dummy_column_names.items()]
    dummy_inv_column_names = list(itertools.chain.from_iterable(dummy_inv_column_names))
    self.dummy_inv_column_names = {k : v for k, v in dummy_inv_column_names}
  
  def transform(self, x):
    res = x
    total_obs = res.shape[0]
    for column_name in self.nominal_feature_list:
      try:
        dummy_values = np.zeros((total_obs, len(self.dummy_map[column_name])))
      except Exception as ex:
        print('Error with column:', column_name)
        raise ex
      for obs, value in enumerate(x[column_name].array):
        if value in self.dummy_map[column_name]:
          dummy_values[obs, self.dummy_map[column_name][value]] = 1
      
      dfDummyValues = pd.DataFrame(dummy_values, columns = self.dummy_column_names[column_name])
      dfDummyValues.reset_index(drop=True, inplace=True)
      res.reset_index(drop=True, inplace=True)

      try:
        res = pd.concat([res, dfDummyValues], axis=1)
      except Exception as ex:
        print('Error with column:', column_name)
        raise ex
    
    res = res.drop(self.nominal_feature_list, axis=1)
    res = res.drop([v for k, v in self.dummy_reference_column.items()], axis=1)

    return res

  def fit_transform(self, x, y=None):
    self.fit(x, y)
    return self.transform(x)

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedKFold
import numpy as np
import lightgbm as lgb
import gc

class FestureImpoertanceVariableSelection(BaseEstimator, TransformerMixin):

  def __init__(self, cummulative_importance_threshold, eval_metric='auc', n_folds=2, early_stopping = True, task_type = 'classification'):
    self.cummulative_importance_threshold = cummulative_importance_threshold
    self.eval_metric = eval_metric
    self.n_folds = n_folds
    if self.n_folds > 1:
      self.skf = StratifiesKFold(n_splits=n_folds)
    self.early_stopping = early_stopping
    self.task_type = task_type
    self.forced_drop_list = []
    self.forced_select_list = []
  
  def set_manual_select_drop_list(self, forced_drop_list = [], forced_select_list = []):
    for v in forced_select_list:
      if v in forced_drop_list:
        raise ValueError('A variable cannot be in select and drop list at the same time')
    
    self.forced_drop_list += forced_drop_list
    self.forced_select_list += forced_select_list
  
  def fit(self, x, y):
    self.nominal_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] == 'category']
    self.interval_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] != 'category']

    dt = DummyTransformer(nominal_feature_list=self.nominal_feature_list)

    #One hot encoding
    features = dt.fit_transform(x)
    self.dt = dt
    self.one_hot_features = [column for column in features.columns if column not in self.nominal_feature_list + self.interval_feature_list]

    #Extract feature names
    feature_names = list(features.columns)

    #Convert to np array
    features = np.array(features)
    labels = np.array(y)

    #Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))

    print('Training Gradient Boosting Model\n')

    if self.n_folds == 1 or not self.early_stopping:

      if self.task_type == 'classification':
        model = lgb.LGBMClassifier(n_estimators=1000, learning_rate = 0.05, verbose = -1)
      elif self.task_type == 'regression':
        model = lgb.LGBMRegressor(n_estimators=1000, learning_rate = 0.05, verbose = -1)
      else:
        raise ValueError('Task must be either "classification" or "regression"')
      
      #Train the model
      model.fit(features, labels)

      #Record the feature importances
      feature_importance_values += model.feature_importances_
    
    else:

      if self.task_type == 'classification':
        model = lgb.LGBMClassifier(n_estimators=1000, learning_rate = 0.05, verbose = -1)
      elif self.task_type == 'regression':
        model = lgb.LGBMRegressor(n_estimators=1000, learning_rate = 0.05, verbose = -1)
      else:
        raise ValueError('Task must be either "classification" or "regression"')
      
      #Iterate through each fold
      for train_index, valid_index in self.skf.split(features, labels):
        train_features = features[train_index]
        valid_features = features[valid_index]
        train_labels = labels[train_index]
        valid_labels = labels[valid_index]

        #Train the model with early stopping
        model.fit(train_features, train_labels, eval_metric = self.eval_metric, eval_set = [(valid_features, valid_labels)], early_stopping_rounds = 100, verbose = -1)

        #Clean up memory
        gc.enable()
        del train_features, train_labels, valid_features, valid_labels
        gc.collect()

        #Record the feature importances
        feature_importance_values += model.feature_importance_
    
    feature_importance_values /= self.n_folds

    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})

    #Add the base
    feature_importances['base_feature'] = feature_importances['feature'].apply(lambda x: dt.dummy_inv_column_names[x] if x in dt.dummy_inv_column_names else x)

    #Sort features according to importance
    feature_importances = feature_importances.sort_values('importance', ascending = False).reset_index(drop = True)

    #Aggregate features by its base form
    feature_importances_grouped = feature_importances.groupby('base_feature')['importance'].sum().to_frame().reset_index().sort_values('importance', ascending = False)

    #Normalize the feature importances to add up to one
    feature_importances_grouped['normalized_importance'] = feature_importances_grouped['importance'] / feature_importances_grouped['importance'].sum()
    feature_importances_grouped['cumulative_importance'] = np.cumsum(feature_importances_grouped['normalized_importance'])
    self.feature_importances = feature_importances_grouped

    #Extract the features with zero importance
    self.zero_importance_features = self.feature_importances[self.feature_importances['importance'] == 0.0]
    self.drop_list_zero_importance = list(self.zero_importance_features['base_feature'])

    #Identify the features not needed to reach the cummulative importance
    record_low_importance = self.feature_importances[self.feature_importances['cumulative_importance'] > self.cummulative_importance_threshold]
    self.drop_list_low_importance = list(record_low_importance['base_feature'])
  
  def transform(self, x):
    drop_list = list(np.unique(self.drop_list_zero_importance + self.drop_list_low_importance + self.forced_drop_list))
    drop_list = [d for d in drop_list if d not in self.forced_select_list]

    res = x.drop(drop_list, axis=1)
    return res
  
  def fit_transform(self, x, y=None):
    self.fit(x, y)
    return self.transform(x)
  
  def plot_feature_importances(self, plot_n = 15, threshold = None):
    if self.feature_importances is None:
      raise NotImplementedError('Feature importances have not been determined. Run `fit`')
  
    #Need to adjust number of features if greater than the features in the data
    if plot_n > self.feature_importances.shape[0]:
      plot_n = self.feature_importances.shape[0] - 1
    
    #Make a horizontal bar chart of feature importances
    plt.figure(figsize = (8, 6))
    ax = plt.subplot()

    #Need to reverse the index to plot most important on top
    ax.barh(np.arange(plot_n),
            self.feature_importances['normalized_feature'][:plot_n],
            align = 'center', edgecolor = 'k')
    
    #Set the yticks and labels
    ax.set_yticks(np.arange(plot_n))
    ax.set_yticklabels(self.feature_importances['base_feature'][:plot_n], size = 10)

    plt.gca().invert_yaxis()

    #Plot labeling
    plt.xlabel('Normalized Importance', size = 16)
    plt.title('Feature Importances', size = 18)
    plt.show()

    #Cumulative importance plot
    plt.figure(figsize = (8, 6))
    plt.plot(list(range(1, len(self.feature_importances) + 1)), self.feature_importances['cumulative_importance'], 'r-')
    plt.xlabel('Number of Features', size = 14)
    plt.ylabel('Cumulative Importance', size = 14)
    plt.title('Cumulative Feature Importance', size = 16)

    if threshold:
      #Index of minimum number of features needed for cumulative importance threshold
      #np.where returns the index so need to add 1 to have correct number
      importance_index = np.min(np.where(self.feature_importances['cumulative_importance'] > threshold))
      plt.vlines(x = importance_index + 1, ymin = 0, ymax = 1, linestyles = '--', colors = 'blue')
      plt.show()

      print('%d features required for %0.2f of cumulative importance' % (importance_index + 1, threshold))

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import gc

class RegularizationFeatureSelection(BaseEstimator, TransformerMixin):

  def __init__(self, n_folds=2, RegularizationStrength = 1.0, task_type = 'classification'):
    self.n_folds = n_folds
    self.RegularizationStrength = RegularizationStrength
    if self.n_folds > 1:
      self.skf = StratifiesKFold(n_splits=n_folds)
    self.task_type = task_type
    self.forced_drop_list = []
    self.forced_select_list = []
  
  def set_manual_select_drop_list(self, forced_drop_list = [], forced_select_list = []):
    for v in forced_select_list:
      if v in forced_drop_list:
        raise ValueError('A variable cannot be in select and drop list at the same time')
    
    self.forced_drop_list += forced_drop_list
    self.forced_select_list += forced_select_list
  
  def fit(self, x, y):
    self.nominal_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] == 'category']
    self.interval_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] != 'category']

    dt = DummyTransformer(nominal_feature_list=self.nominal_feature_list)

    #One hot encoding
    features = dt.fit_transform(x)
    self.dt = dt
    self.one_hot_features = [column for column in features.columns if column not in self.nominal_feature_list + self.interval_feature_list]

    fold_selected_features = np.zeros((self.n_folds, features.shape[1]), dtype=bool)
    fold_selected_features_coefs = np.zeros((self.n_folds, features.shape[1]), dtype=float)

    #Extract feature names
    feature_names = features.columns

    #Convert to np array
    features = np.array(features)
    labels = np.array(y)

    print('Training Lasso Regression Model\n')

    if self.n_folds > 1:
      #Iterate through each fold
      for train_index, valid_index in self.skf.split(features, labels):
        scaler = StandardScaler()
        if self.task_type == 'classification':
          model = SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear', C=self.RegularizationStrength, class_weight = 'balanced'))
        elif self.task_type == 'regression':
          model = SelectFromModel(Lasso(alpha=self.RegularizationStrength))
        else:
          raise ValueError('Task type must be either "classification" or "regression"')

        train_features = features[train_index]
        train_labels = labels[train_index]

        #Train the model
        scaled_features = scaler.fit_transform(train_features)
        model.fit(scaled_features, train_labels)
    
        #Record the feature importances
        fold_selected_features[self.n_folds-1,:] = model.get_support()
        fold_selected_features_coefs[self.n_folds-1,:] = model.estimator_.coef_

        #Clean up memory
        gc.enable()
        del train_features, train_labels
        gc.collect()

    else:
      scaler = StandardScaler()
      if self.task_type == 'classification':
        model = SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear', C=self.RegularizationStrength, class_weight = 'balanced'))
      elif self.task_type == 'regression':
        model = SelectFromModel(Lasso(alpha=self.RegularizationStrength))
      else:
        raise ValueError('Task type must be either "classification" or "regression"')
      scaled_features = scaler.fit_transform(features)
      model.fit(scaled_features, y)

      #Record the feature importances
      fold_selected_features[self.n_folds-1,:] = model.get_support()
      fold_selected_features_coefs[self.n_folds-1,:] = model.estimator_.coef_
  
    selected_features = np.sum(fold_selected_features, axis = 0, dtype = bool)
    selected_features_coefs = np.mean(fold_selected_features_coefs, axis = 0, dtype = float)

    self.selected_features = feature_names[selected_features]
    self.selected_features_averaged_coefs = selected_features_coefs[selected_features]
    list_zero_coeficient = feature_names[selected_features == False]

    interval_list_zero_coeficient = list(np.array([v for v in list_zero_coeficient if v not in dt.dummy_inv_column_names]))
    nominal_list_zero_coeficient = list(np.unique([dt.dummy_inv_column_names[v] for v in list_zero_coeficient if v in dt.dummy_inv_column_names]))
    nominal_list_nonzero_coeficient = list(np.unique([dt.dummy_inv_column_names[v] for v in self.selected_features if v in dt.dummy_inv_column_names]))
    
    self.drop_list_zero_coeficient = interval_list_zero_coeficient + [v for v in nominal_list_zero_coeficient if v not in nominal_list_nonzero_coeficient]

    dfNonZeroAbsoluteCoefs = pd.DataFrame(list(zip(self.selected_features, self.selected_features_averaged_coefs, np.abs(self.selected_features_averaged_coefs))), columns=['feature', 'coef', 'absolute_coef'])
    dfZeroAbsoluteCoefs = pd.DataFrame(list(zip(self.drop_list_zero_coeficient, [0.0] * len(self.drop_list_zero_coeficient), [0.0] * len(self.drop_list_zero_coeficient))), columns=['feature', 'coef', 'absolute_coef'])
    dfAbsoluteCoefs = dfNonZeroAbsoluteCoefs.append(dfZeroAbsoluteCoefs, ignore_index=True)
    self.dfAbsoluteCoefs = dfAbsoluteCoefs.sort_values('absolute_coef', ascending = False)

  def transform(self, x):
    drop_list = list(np.unique(self.drop_list_zero_coeficient + self.forced_drop_list))
    drop_list = [d for d in drop_list if d not in self.forced_select_list]

    res = x.drop(drop_list, axis=1)
    return res
  
  def fit_transform(self, x, y=None):
    self.fit(x, y)
    return self.transform(x)
  
  def plot_feature_coefs(self, plot_n = 15, threshold = None):
    if self.dfAbsoluteCoefs is None:
      raise NotImplementedError('Feature coefs have not been determined. Run `fit`')
  
    #Need to adjust number of features if greater than the features in the data
    if plot_n > self.dfAbsoluteCoefs.shape[0]:
      plot_n = self.dfAbsoluteCoefs.shape[0] - 1

    colors = ['b' if c > 0.0 else 'r' for c in list(self.dfAbsoluteCoefs['coef'])]
    
    #Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()

    #Need to reverse the index to plot most important on top
    ax.barh(np.arange(plot_n),
            self.dfAbsoluteCoefs['coef'][:plot_n],
            align = 'center', edgecolor = 'k', color=colors)

    plt.gca().invert_yaxis()

    #Set the yticks and labels
    ax.set_yticks(np.arange(plot_n))
    ax.set_yticklabels(self.dfAbsoluteCoefs['feature'][:plot_n], size = 12)

    #Plot labeling
    plt.xlabel('Coeficient', size = 16)
    plt.title('Feature Coeficients', size = 18)
    plt.show()

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns

class VariableClusteringSelection(BaseEstimator, TransformerMixin):

  def __init__(self, n_folds=2, dissimilarity_threshold = 0.5, use_class_features = False):
    self.dissimilarity_threshold = dissimilarity_threshold
    self.use_class_features = use_class_features
    self.forced_drop_list = []
    self.forced_select_list = []

  def set_manual_select_drop_list(self, forced_drop_list = [], forced_select_list=[]):
    for v in forced_select_list:
      if v in forced_drop_list:
        raise ValueError('A variable cannot be in select and drop list at the same time')
    
    self.forced_drop_list += forced_drop_list
    self.forced_select_list += forced_select_list
  
  def fit(self, x, y=None):
    self.nominal_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] == 'category']
    self.interval_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] != 'category']

    dt = DummyTransformer(nominal_feature_list=self.nominal_feature_list)

    if self.use_class_features:
      #One hot encoding
      features = dt.fit_transform(x)
      self.dt = dt
      self.one_hot_features = [column for column in features.columns if column not in self.nominal_feature_list + self.interval_feature_list]
    else:
      features = x[self.interval_feature_list]
      self.set_manual_select_drop_list(forced_select_list = self.nominal_feature_list)
    
    #correlation matrix
    corr_matrix = features.corr()

    #dissimilarity matrix
    dissimilarity_matrix = 1 - np.abs(corr_matrix)

    #variable clustering
    print('Fitting Agglomerative Clustering\n')
    clustering = AgglomerativeClustering(linkage='ward', n_clusters=None, distance_threshold = self.dissimilarity_threshold)
    clustering.fit(dissimilarity_matrix)

    #cluster proximities
    clusters_proximities = []
    for cluster_row in np.sort(np.unique(clustering.labels_)):
      for cluster_column in range(cluster_row + 1, np.max(clustering.labels_) + 1):
        clusters_proximities.append([cluster_row, cluster_column,
                                     np.mean(dissimilarity_matrix.loc[dissimilarity_matrix.columns[np.where(clustering.labels_ == cluster_row)[0]]].values)])
        clusters_proximities.append([cluster_column, cluster_row,
                                     np.mean(dissimilarity_matrix.loc[dissimilarity_matrix.columns[np.where(clustering.labels_ == cluster_row)[0]]].values)])
    
    df_clusters_proximities = pd.DataFrame(clusters_proximities, columns=['cluster', 'neighbor', 'dissimilarity'])

    #nearest cluster
    df_nearest_cluster = df_clusters_proximities.sort_values('dissimilarity').groupby('cluster').head(1)

    #variable scores
    variable_scores = []

    for ix, row in df_nearest_cluster.iterrows(): #foreach cluster
      cluster_variables = np.where(clustering.labels_ == row['cluster'])[0] #own clusters variables
      neighbor_variables = np.where(clustering.labels_ == row['neighbor'])[0] #nearest clusters variables

      #if one-variable cluster
      if len(cluster_variables) == 1:
        Y_corr = features[dissimilarity_matrix.columns[cluster_variables[0]]]
        X_corr = features[dissimilarity_matrix.columns[neighbor_variables]]
        reg = LinearRegression().fit(X_corr, Y_corr)
        neighbor_score = reg.score(X_corr, Y_corr)

        variable_scores.append([row['cluster'], dissimilarity_matrix.columns[cluster_variables[0]], 1.0, neighbor_score, (1 - 1)/(1 - neighbor_score)])
      else:
        for v in cluster_variables:
          Y_corr = features[dissimilarity_matrix.columns[v]]
          X_corr = features[dissimilarity_matrix.columns[cluster_variables[np.where(np.array(cluster_variables) != v)]]]
          reg = LinearRegression().fit(X_corr, Y_corr)
          cluster_score = reg.score(X_corr, Y_corr)

          Y_corr = features[dissimilarity_matrix.columns[v]]
          X_corr = features[dissimilarity_matrix.columns[neighbor_variables]]
          reg = LinearRegression().fit(X_corr, Y_corr)
          neighbor_score = reg.score(X_corr, Y_corr)

          variable_scores.append([row['cluster'], dissimilarity_matrix.columns[v], cluster_score, neighbor_score, (1 - cluster_score)/(1 - neighbor_score)])
    
    df_variable_scores = pd.DataFrame(variable_scores, columns=['cluster', 'variable', 'own_cluster_Rˆ2', 'neighbor_cluster_Rˆ2', '1-Rˆ2 Ratio'])

    df_best_variables = df_variable_scores.sort_values('1-Rˆ2 Ratio').groupby('cluster').head(1)

    #results
    self.corr_matrix = corr_matrix
    self.clustering = clustering
    self.df_clusters_proximities = df_clusters_proximities
    self.df_nearest_cluster = df_nearest_cluster
    self.df_variable_scores = df_variable_scores
    self.df_best_variables = df_best_variables
    self.select_list_best_variables = list(df_best_variables['variable'].values)
  
  def transform(self, x):
    if self.use_class_features:
      #One hot encoding
      features = self.dt.transform(x)
    else:
      features = x
    
    res = features[np.unique(self.select_list_best_variables + self.forced_select_list)]
    res = res.drop(self.forced_drop_list, axis=1)
    return res
  
  def fit_transform(self, x, y=None):
    self.fit(x, y)
    return self.transform(x)
  
  def plot_collinear(self, correlation_threshold = 0.7, cmap='bwr'):
    """
    Adapted from reference: https://github.com/WillKoehrsen/feature-selector/blob/master/feature_selector/feature_selector.py
    """
    upper = self.corr_matrix.where(np.triu(np.ones(self.corr_matrix.shape), k = 1).astype(np.bool))

    #Dataframe to hold correlated pairs
    record_collinear = pd.DataFrame(columns = ['feature', 'corr_feature', 'corr_value'])

    for c in upper.columns:
      #Find the correlated features
      corr_features = list(upper.index[upper[c].abs() > correlation_threshold])
      if len(corr_features) == 0:
        continue
      #Find the correlated features
      corr_values = list(upper[c][upper[c].abs() > correlation_threshold])
      features = [c for _ in range(len(corr_features))]

      #Record the information (need a temp df for now)
      temp_df = pd.DataFrame.from_dict({'feature': features,
                                        'corr_feature': corr_features,
                                        'corr_value': corr_values})
      
      #Add to dataframe
      record_collinear = record_collinear.append(temp_df, ignore_index = True)
    
    record_collinear.sort_values(['feature', 'corr_value'], ascending=[True, False])

    #Identify the correlations that were above the threshold
    #columns (x-axis) are features to drop and rows (y-axis) are correlated pairs
    corr_matrix_plot = self.corr_matrix.loc[list(set(record_collinear['corr_feature'])),
                                            list(set(record_collinear['feature']))]
    
    if len(corr_matrix_plot) == 0:
      print('No correlation above threshold')
      return
    
    corr_matrix_plot = corr_matrix_plot.sort_values(list(corr_matrix_plot.columns)).sort_values(list(corr_matrix_plot.index), axis=1)
    self.corr_matrix_plot = corr_matrix_plot

    title = "Reduced Correlation Above Threshold Matrix"

    f, ax = plt.subplots(figsize=(10, 8))

    #Draw the heatmap with a color bar
    sns.heatmap(corr_matrix_plot, center=0, cmap=cmap,
                linewidths=.25, cbar_kws={"shrink": 0.6})
    
    #Set the ylabels
    ax.set_yticks([x + 0.5 for x in list(range(corr_matrix_plot.shape[0]))])
    ax.set_yticklabels(list(corr_matrix_plot.index), size = int(160 / corr_matrix_plot.shape[0]))

    #Set the xlabels
    ax.set_xticks([x + 0.5 for x in list(range(corr_matrix_plot.shape[1]))])
    ax.set_xticklabels(list(corr_matrix_plot.columns), size = int(160 / corr_matrix_plot.shape[1]))

    plt.title(title, size=14)

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

class OptimalNominalBinning(BaseEstimator, TransformerMixin):

  def __init__(self, sigLevel, rare_cutoff_percentage=0.005, balanced=False, verbose= 0, ignore_feature_list = [], max_bins = 10, task_type = 'classification'):
    self.sigLevel = sigLevel
    self.rare_cutoff_percentage = rare_cutoff_percentage
    self.balanced = balanced
    self.verbose = verbose
    self.ignore_feature_list = ignore_feature_list
    self.task_type = task_type
    self.max_bins = max_bins
    self.map = {}
    self.original_map = {}
    self.inv_map = {}
    self.original_inv_map = {}
    self.statistics = {}
    self.original_statistics = {}
  
  def fit(self, x, y):
    self.x = x
    self.y = y
    self.targetColumn = y.columns[0]
    series = [x, y[self.targetColumn].reset_index(drop = True)]
    self.AllData = pd.concat(series, axis=1)

    self.nominal_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] == 'category' or '_dummy_' in k or k[:5] in ('cbin_', 'ibin_')]
    self.nominal_feature_list = [k for k in self.nominal_feature_list if k not in self.ignore_feature_list]
    self.interval_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] != 'category' and '_dummy_'not in k and k[:5] not in ('cbin_', 'ibin_')]

    for feature in x[self.nominal_feature_list].columns:
      dfBinFeature = self.optimalClassGrouping(x[self.nominal_feature_list], y, feature)
      self.map[feature] = {k : v for k, v in dfBinFeature.drop_duplicates([feature, 'bin'])[[feature, 'bin']].values}
      self.original_map[feature] = self.map[feature].copy()

      self.inv_map[feature] = {}
      for v, b in self.map[feature].items():
        if b not in self.inv_map[feature]:
          self.inv_map[feature][b] = [v]
        else:
          self.inv_map[feature][b].append(v)
      
      self.original_inv_map[feature] = self.inv_map[feature].copy()

      self.statistics[feature] = pd.DataFrame([k for k, v in self.inv_map[feature].items()], columns=['bin']).sort_values('bin').reset_index(drop=True)

      def bin_count(x):
        b = x
        return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].count()

      def bin_mean(x):
        b = x
        return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].mean()

      def bin_sum(x):
        b = x
        return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].sum()
      
      def get_class_list(x):
        if x not in self.inv_map[feature]:
          raise ValueError('bin ' + str(x) + ' does not exist for the feature ' + str(feature))
        return str(self.inv_map[feature][x])

      self.statistics[feature]['n_obs'] = self.statistics[feature]['bin'].apply(bin_count)
      self.statistics[feature]['n_events'] = self.statistics[feature]['bin'].apply(bin_sum)
      self.statistics[feature]['event_rate'] = self.statistics[feature]['bin'].apply(bin_mean)
      self.statistics[feature]['class_list'] = self.statistics[feature]['bin'].apply(get_class_list)

      self.original_statistics[feature] = self.statistics[feature].copy()

  def transform(self, x):
    features = x

    for feature in self.x[self.nominal_feature_list].columns:
      if feature in self.ignore_feature_list:
        continue
      
      for v in np.unique(self.x[feature].values):
        if v not in self.map[feature]:
          print('column', feature, 'Value:', v)
          raise NotImplementedError('Valur out of the bounds')
      features['cbin_' + feature] = features[feature].apply(lambda x: self.map[feature][x] if x in self.map[feature] else None)
      features = features.drop([feature], axis=1)
    
    return features
  
  def ignore_feature(self, feature):
    self.ignore_feature_list.append(feature)
  
  def reset_feature(self, feature):
    if feature not in self.x[self.nominal_feature_list].columns:
      raise NotImplementedError('Referenced feature does not exists in trained DataFrame')
    self.map[feature] = self.original_map[feature]
    self.inv_map[feature] = self.original_inv_map[feature]
    self.statistics[feature] = self.original_statistics[feature]
  
  def changeFeatureBin(self, feature, feature_value, bin_value):
    if feature not in self.x[self.nominal_feature_list].columns:
      raise NotImplementedError('Referenced feature does not exists in trained DataFrame')
    if feature not in self.map[feature]:
      raise NotImplementedError('Referenced value does not exists in feature domain')
    self.map[feature][feature_value] = bin_value

    self.inv_map[feature] = {}
    for v, b in self.map[feature].items():
      if b not in self.inv_map[feature]:
        self.inv_map[feature][b] = [v]
      else:
        self.inv_map[feature][b].append(v)
    
    self.original_inv_map[feature] = self.inv_map[feature].copy()

    self.statistics[feature] = pd.DataFrame([k for k, v in self.inv_map[feature].items()], columns=['bin']).sort_values('bin').reset_index(drop=True)

    def bin_count(x):
      b = x
      return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].count()

    def bin_mean(x):
      b = x
      return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].mean()

    def bin_sum(x):
      b = x
      return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].sum()
    
    def get_class_list(x):
      if x not in self.inv_map[feature]:
        raise ValueError('bin ' + str(x) + ' does not exist for the feature ' + str(feature))
      return str(self.inv_map[feature][x])

    self.statistics[feature]['n_obs'] = self.statistics[feature]['bin'].apply(bin_count)
    self.statistics[feature]['n_events'] = self.statistics[feature]['bin'].apply(bin_sum)
    self.statistics[feature]['event_rate'] = self.statistics[feature]['bin'].apply(bin_mean)
    self.statistics[feature]['class_list'] = self.statistics[feature]['bin'].apply(get_class_list)

  def moveBin(self, feature, current_bin_value, next_bin_value):
    if feature not in self.x[self.nominal_feature_list].columns:
      raise NotImplementedError('Referenced feature does not exists in trained DataFrame')
    if feature not in self.map[feature]:
      raise NotImplementedError('Referenced value does not exists in feature domain')
    for k, v in self.map[feature].items():
      if current_bin_value == v:
        self.map[feature][k] = next_bin_value

    self.inv_map[feature] = {}
    for v, b in self.map[feature].items():
      if b not in self.inv_map[feature]:
        self.inv_map[feature][b] = [v]
      else:
        self.inv_map[feature][b].append(v)
    
    self.original_inv_map[feature] = self.inv_map[feature].copy()

    self.statistics[feature] = pd.DataFrame([k for k, v in self.inv_map[feature].items()], columns=['bin']).sort_values('bin').reset_index(drop=True)

    def bin_count(x):
      b = x
      return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].count()

    def bin_mean(x):
      b = x
      return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].mean()

    def bin_sum(x):
      b = x
      return self.AllData[self.AllData[feature].isin(self.inv_map[feature][b])][self.targetColumn].sum()
    
    def get_class_list(x):
      if x not in self.inv_map[feature]:
        raise ValueError('bin ' + str(x) + ' does not exist for the feature ' + str(feature))
      return str(self.inv_map[feature][x])

    self.statistics[feature]['n_obs'] = self.statistics[feature]['bin'].apply(bin_count)
    self.statistics[feature]['n_events'] = self.statistics[feature]['bin'].apply(bin_sum)
    self.statistics[feature]['event_rate'] = self.statistics[feature]['bin'].apply(bin_mean)
    self.statistics[feature]['class_list'] = self.statistics[feature]['bin'].apply(get_class_list)
  
  def fit_transform(self, x, y):
    self.fit(x, y)
    return self.transform(x)
  
  def optimalClassGrouping(self, dfFeatures, dfTarget, feature):

    if feature not in dfFeatures.columns:
      raise NotImplementedError('rankColumn feature does not exists in trained DataFrame')
    if len(dfFeatures) != len(dfTarget):
      raise NotImplementedError('feature and target DataFrames must have same length')

    targetColumn = dfTarget.columns[0]

    series = [dfFeatures[feature], dfTarget[targetColumn].reset_index(drop=True)]
    dfOriginalData = pd.concat(series, axis=1)

    if self.balanced:
      dfOriginalDataY1 = dfOriginalData[dfOriginalData[targetColumn] == 1]
      dfOriginalData = dfOriginalDataY1.append(dfOriginalData[dfOriginalData[targetColumn] == 0].sample(len(dfOriginalDataY1)), ignore_index = True)
    dfDerivedData = dfOriginalData

    dfDerivedData['countColumn'] = 1

    dfGroupRareClasses = dfDerivedData.groupby(feature).aggregate({'countColumn' : np.sum, targetColumn : np.mean}).reset_index()
    dfGroupRareClasses['FgRare'] = dfGroupRareClasses['countColumn'].apply(lambda x: 1 if x / sum(dfGroupRareClasses['countColumn'].values) < self.rare_cutoff_percentage else 0)
    dfGroupRareClasses.reset_index(level=0, inplace=True)
    dfGroupRareClasses['CdGroup'] = dfGroupRareClasses.apply(lambda x: x['index'] if x['FgRare'] == 0 else np.min(dfGroupRareClasses[dfGroupRareClasses['FgRare'] == 1]['index'].values), axis=1)
    dictCdGroup = {k : v for k, v in dfGroupRareClasses[[feature, 'CdGroup']].values}

    dfStatistics = dfGroupRareClasses.groupby('CdGroup').aggregate({'countColumn' : np.sum, targetColumn : np.mean}).sort_values(targetColumn, ascending=False).reset_index()
    dfStatistics.reset_index(level=0, inplace=True)
    dictBin = {k:v for k, v in dfStatistics[['CdGroup', 'index']].values}

    dfDerivedData['bin'] = dfDerivedData[feature].apply(lambda x: dictBin[dictCdGroup[x]]).astype('int64')

    print(feature)
    print(dfDerivedData.groupby('bin').aggregate({'countColumn' : np.sum, targetColumn : [np.sum, np.mean]}))

    flgFinished = False
    while(not flgFinished):
      flgUpdated = False

      bins = dfDerivedData.bin.unique()
      bins.sort()

      if len(bins) == 1:
        print('Process finished')
        return dfDerivedData
      
      bin_comparision = np.zeros((len(bins)-1, 7))
      for b in bins[:-1]:
        n_current_bin = dfDerivedData[(dfDerivedData.bin == b)].shape[0]
        p_current_bin = dfDerivedData[(dfDerivedData.bin == b)][targetColumn].mean()

        n_next_bin = dfDerivedData[(dfDerivedData.bin == bins[np.where(bins == b)[0][0]+1])].shape[0]
        p_next_bin = dfDerivedData[(dfDerivedData.bin == bins[np.where(bins == b)[0][0]+1])][targetColumn].mean()

        p_hat = ((n_current_bin*p_current_bin)+(n_next_bin*p_next_bin)) / (n_current_bin+n_next_bin)
        q_hat = 1 - p_hat
        std_error = ((p_hat*q_hat)*(n_current_bin+n_next_bin)/(n_current_bin*n_next_bin)) ** 0.5

        if std_error == .0:
          z_value = 1
        else:
          try:
            z_value = (p_current_bin - p_next_bin)/std_error
          except:
            print(b, n_current_bin, p_current_bin, n_next_bin, p_next_bin, p_hat, q_hat)
            z_value = (p_current_bin - p_next_bin)/std_error
        
        p_value = scipy.stats.norm.sf(abs(z_value)) * 2

        bin_comparision[np.where(bins == b)[0][0]] = [b, bins[np.where(bins == b)[0][0]+1] if p_value > self.sigLevel else b, p_value, n_current_bin, n_next_bin, p_current_bin, p_next_bin]

        flgUpdated = True if p_value > self.sigLevel else flgUpdated
      
      #if not flgUpdated:
      #  print('Process finished')
      #  return dfDerivedData

      if not flgUpdated and len(bins) <= self.max_bins:
        print('Process finished')
        return dfDerivedData
      
      df_bin_comparision = pd.DataFrame(bin_comparision, columns=['bin', 'next_bin' ,'p_value', 'n_current_bin', 'n_next_bin', 'p_current_bin', 'p_next_bin'])

      #dfH0 = df_bin_comparision[df_bin_comparision.p_value > seg.sigLevel].sort_values(by='p_value', ascending=False)
      dfH0 = df_bin_comparision.sort_values(by='p_value', ascending=False)
      #dfH0_max = dfH0.ix[dfH0['p_value'].idmax()]
      dfH0_max = dfH0.loc[dfH0['p_value'].idxmax()]

      current_bin = dfH0_max.bin
      next_bin = dfH0_max.next_bin
      p_value = dfH0_max.p_value
      n_current_bin = dfH0_max.n_current_bin
      n_next_bin = dfH0_max.n_next_bin
      p_current_bin = dfH0_max.p_current_bin
      p_next_bin = dfH0_max.p_next_bin

      print('current:', current_bin, 'next_bin:', next_bin ,'p_value:', p_value, 'n_current_bin:', n_current_bin, 'n_next_bin', 'p_current_bin:', p_current_bin, 'p_next_bin:', p_next_bin)

      print(current_bin, ' now is', next_bin)
      dfDerivedData.loc[dfDerivedData['bin'] == current_bin, 'bin'] = next_bin
      dfDerivedData['bin'] = dfDerivedData['bin'].astype('int64')

      print(dfDerivedData.groupby('bin').aggregate({'countColumn' : np.sum, targetColumn : [np.sum, np.mean]}))
    
    return dfDerivedData.sort_values('bin').reset_index(drop=True)

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

class OptimalIntervalBinning(BaseEstimator, TransformerMixin):

  def __init__(self, sigLevel, n_rank_groups, balanced=False, verbose=0, max_bins =10, ignore_feature_list = [], task_type = 'classification'):
    self.sigLevel = sigLevel
    self.n_rank_groups = n_rank_groups
    self.balanced = balanced
    self.verbose = verbose
    self.max_bins = max_bins
    self.ignore_feature_list = ignore_feature_list
    self.task_type = task_type
    self.ranges = {}
    self.original_ranges = {}
    self.statistics = {}
    self.original_statistics = {}
  
  def fit(self, x, y):
    self.x = x
    self.y = y
    self.targetColumn = y.columns[0]
    series = [x, y[self.targetColumn].reset_index(drop = True)]
    self.AllData = pd.concat(series, axis=1)

    self.nominal_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] == 'category' or '_dummy_' in k or k[:5] in ('cbin_', 'ibin_')]
    self.nominal_feature_list = [k for k in self.nominal_feature_list if k not in self.ignore_feature_list]
    self.interval_feature_list = [k for k, v in x.dtypes.to_dict().items() if v.name[:len('category')] != 'category' and '_dummy_'not in k and k[:5] not in ('cbin_', 'ibin_')]

    for feature in x[self.interval_feature_list].columns:
      dfBinFeature = self.optimalPercentileGrouping(x[self.interval_feature_list], y, feature)
      initial_statistics = dfBinFeature.groupby('bin').aggregate({feature: [np.min, np.max], 'countColumn' : np.sum, self.targetColumn : [np.sum, np.mean]})
      bondaries = initial_statistics.values[:,1]
      bins = initial_statistics.index.values

      first_bin = bins[0]
      last_bin = bins[-1]
      

      first_bondary = -float('inf')
      last_bondary = float('inf')
      previous_split = -float('inf')
      ranges = []
      n_range = 0

      if first_bin == last_bin:
        ranges.append([n_range, first_bondary, last_bondary])
      else:
        prev_last_bin = bins[-2]
        for current_bin, current_split in zip(bins[:-1], bondaries[:-1]):
          if current_bin == first_bin:
            if first_bin == prev_last_bin:
              ranges.append([n_range, previous_split, last_bondary])
            else:
              continue
          else:
            ranges.append([n_range, previous_split, current_split])
            if current_bin == prev_last_bin:
              ranges.append([n_range+1, current_split, last_bondary])
          previous_split = current_split
          n_range += 1
      ranges = np.array(ranges)

      self.ranges[feature] = ranges
      try:
        self.statistics[feature] = pd.DataFrame(ranges, columns=['n_range', 'lower_bondary', 'upper_bondary'])
      except Exception as ex:
        print('Error with feature:', feature, ranges)
        raise ex

      def range_count(x):
        lower_bondary = x[0]
        upper_bondary = x[1]
        return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].count()

      def range_sum(x):
        lower_bondary = x[0]
        upper_bondary = x[1]
        return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].sum()

      def range_mean(x):
        lower_bondary = x[0]
        upper_bondary = x[1]
        return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].mean()

      self.statistics[feature]['n_obs'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_count, axis=1)
      self.statistics[feature]['n_events'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_sum, axis=1)
      self.statistics[feature]['event_rate'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_mean, axis=1)

      self.original_ranges[feature] = self.ranges[feature].copy()
      self.original_statistics[feature] = self.statistics[feature].copy()

  def transform(self, x):
    features = x

    for feature in self.x[self.interval_feature_list].columns:
      if feature in self.ignore_feature_list:
        continue
      
      def get_ix_range(v):
        for new_n_range, lowerBondary, upperBondary in self.ranges[feature]:
          if v >= lowerBondary and v < upperBondary:
            return new_n_range
        print('feature:', feature, 'value:', v)
        raise NotImplementedError('Value out of the bounds')
      features['ibin_' + feature] = features[feature].apply(get_ix_range)
      features = features.drop([feature], axis=1)
    
    return features
  
  def ignore_feature(self, feature):
    self.ignore_feature_list.append(feature)
  
  def reset_feature(self, feature):
    if feature not in self.x[self.interval_feature_list].columns:
      raise NotImplementedError('Referenced feature does not exists in trained DataFrame')
    self.ranges[feature] = self.original_ranges[feature]
    self.statistics[feature] = self.original_statistics[feature]
  
  def splitRange(self, feature, split_range, split_value):
    new_ranges = []
    new_n_range = 0

    if feature not in self.x[self.interval_feature_list].columns:
      raise NotImplementedError('Referenced feature does not exists in trained DataFrame')
    if split_range not in self.ranges[feature][:,0]:
      raise NotImplementedError('Referenced range does not exists in range list')
    
    for n_range, lowerBondary, upperBondary in self.ranges[feature]:
      if split_range == n_range:
        if not (split_value >= lowerBondary and split_value < upperBondary):
          raise NotImplementedError('Split value for range ' + str(split_range) + ' must be greater or equal than' + str(lowerBondary) + ' and less than ' + str(upperBondary))
        new_ranges.append([new_n_range, lowerBondary, split_value])
        new_n_range += 1
        new_ranges.append([new_n_range, split_value, upperBondary])
        new_n_range += 1
      else:
        new_ranges.append([new_n_range, lowerBondary, upperBondary])
        new_n_range += 1

    new_ranges = np.array(new_ranges)
    self.ranges[feature] = new_ranges

    self.statistics[feature] = pd.DataFrame(new_ranges, columns=[['n_range', 'lower_bondary', 'upper_bondary']])

    def range_count(x):
      lower_bondary = x[0]
      upper_bondary = x[1]
      return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].count()

    def range_sum(x):
      lower_bondary = x[0]
      upper_bondary = x[1]
      return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].sum()

    def range_mean(x):
      lower_bondary = x[0]
      upper_bondary = x[1]
      return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].mean()

    self.statistics[feature]['n_obs'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_count, axis=1)
    self.statistics[feature]['n_events'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_sum, axis=1)
    self.statistics[feature]['event_rate'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_mean, axis=1)

  def mergeRange(self, feature, new_start_range, new_end_range):
    new_ranges = []
    new_n_range = 0
    flg_start_new_range = False
    first_lowerBondary = -float('inf')

    if feature not in self.x[self.interval_feature_list].columns:
      raise NotImplementedError('Referenced feature does not exists in trained DataFrame')
    if new_start_range not in self.ranges[feature][:,0]:
      raise NotImplementedError('Referenced start range does not exists in range list')
    if new_end_range not in self.ranges[feature][:,0]:
      raise NotImplementedError('Referenced end range does not exists in range list')
    
    for n_range, lowerBondary, upperBondary in self.ranges[feature]:
      if new_start_range == n_range:
        first_lowerBondary = lowerBondary
        flg_start_new_range = True
      elif new_end_range == n_range:
        flg_start_new_range = False
        new_ranges.append([new_n_range, first_lowerBondary, upperBondary])
        new_n_range += 1
      elif flg_start_new_range:
        continue
      else:
        new_ranges.append([new_n_range, lowerBondary, upperBondary])
        new_n_range += 1
    
    new_ranges = np.array(new_ranges)  
    self.ranges[feature] = new_ranges

    self.statistics[feature] = pd.DataFrame(new_ranges, columns=[['n_range', 'lower_bondary', 'upper_bondary']])

    def range_count(x):
      lower_bondary = x[0]
      upper_bondary = x[1]
      return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].count()

    def range_sum(x):
      lower_bondary = x[0]
      upper_bondary = x[1]
      return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].sum()

    def range_mean(x):
      lower_bondary = x[0]
      upper_bondary = x[1]
      return self.AllData[(self.AllData[feature] >= lower_bondary) & (self.AllData[feature] < upper_bondary)][self.targetColumn].mean()

    self.statistics[feature]['n_obs'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_count, axis=1)
    self.statistics[feature]['n_events'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_sum, axis=1)
    self.statistics[feature]['event_rate'] = self.statistics[feature][['lower_bondary', 'upper_bondary']].apply(range_mean, axis=1)
  
  def fit_transform(self, x, y):
    self.fit(x, y)
    return self.transform(x)
  
  def optimalPercentileGrouping(self, dfFeatures, dfTarget, feature):

    if feature not in dfFeatures.columns:
      raise NotImplementedError('rankColumn feature does not exists in trained DataFrame')
    if len(dfFeatures) != len(dfTarget):
      raise NotImplementedError('feature and target DataFrames must have same length')

    rankColumn = 'rank_' + feature
    targetColumn = dfTarget.columns[0]
    featureDomain = np.unique(dfFeatures[feature].values)

    series = [dfFeatures[feature], dfTarget[targetColumn].reset_index(drop=True)]
    dfOriginalData = pd.concat(series, axis=1)

    if self.balanced:
      dfOriginalDataY1 = dfOriginalData[dfOriginalData[targetColumn] == 1]
      dfOriginalData = dfOriginalDataY1.append(dfOriginalData[dfOriginalData[targetColumn] == 0].sample(len(dfOriginalDataY1)), ignore_index = True)
    dfDerivedData = dfOriginalData

    if len(featureDomain) > self.n_rank_groups:
      dfDerivedData[rankColumn] = (dfDerivedData[feature].rank(pct=True)*self.n_rank_groups).round().rename(rankColumn).reset_index(drop=True)
    else:
      dfDerivedData[rankColumn] = dfDerivedData[feature].rank(method='dense').rename(rankColumn).reset_index(drop=True)

    dfDerivedData['bin'] = dfDerivedData[rankColumn].astype('int64')
    dfDerivedData['countColumn'] = 1

    print(feature)

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
      print(dfDerivedData.groupby('bin').aggregate({'countColumn' : np.sum, targetColumn : [np.sum, np.mean], feature : [np.min, np.max]}))

    flgFinished = False
    while(not flgFinished):
      flgUpdated = False

      bins = dfDerivedData.bin.unique()
      bins.sort()

      if len(bins) == 1:
        print('Process finished')
        return dfDerivedData
      
      bin_comparision = np.zeros((len(bins)-1, 7))
      for b in bins[:-1]:
        n_current_bin = dfDerivedData[(dfDerivedData.bin == b)].shape[0]
        p_current_bin = dfDerivedData[(dfDerivedData.bin == b)][targetColumn].mean()

        n_next_bin = dfDerivedData[(dfDerivedData.bin == bins[np.where(bins == b)[0][0]+1])].shape[0]
        p_next_bin = dfDerivedData[(dfDerivedData.bin == bins[np.where(bins == b)[0][0]+1])][targetColumn].mean()

        p_hat = ((n_current_bin*p_current_bin)+(n_next_bin*p_next_bin)) / (n_current_bin+n_next_bin)
        q_hat = 1 - p_hat
        std_error = ((p_hat*q_hat)*(n_current_bin+n_next_bin)/(n_current_bin*n_next_bin)) ** 0.5

        if std_error == .0:
          z_value = 1
        else:
          try:
            z_value = (p_current_bin - p_next_bin)/std_error
          except:
            print(b, n_current_bin, p_current_bin, n_next_bin, p_next_bin, p_hat, q_hat)
            z_value = (p_current_bin - p_next_bin)/std_error
        
        p_value = scipy.stats.norm.sf(abs(z_value)) * 2

        bin_comparision[np.where(bins == b)[0][0]] = [b, bins[np.where(bins == b)[0][0]+1] if p_value > self.sigLevel or len(bins) > self.max_bins else b, p_value, n_current_bin, n_next_bin, p_current_bin, p_next_bin]

        flgUpdated = True if p_value > self.sigLevel else flgUpdated
      
      if not flgUpdated and len(bins) <= self.max_bins:
        print('Process finished')
        return dfDerivedData
      
      df_bin_comparision = pd.DataFrame(bin_comparision, columns=['bin', 'next_bin' ,'p_value', 'n_current_bin', 'n_next_bin', 'p_current_bin', 'p_next_bin'])

      #Adicionar lógica de número máximo de grupos no anterior
      dfH0 = df_bin_comparision.sort_values(by='p_value', ascending=False)
      #dfH0_max = dfH0.ix[dfH0['p_value'].idmax()]
      dfH0_max = dfH0.loc[dfH0['p_value'].idxmax()]

      #with pd.option_context('display.max_rows', None, 'display.max_columns', None):
      #  print('dfH0:', dfH0)
      #print('dfH0_max:', dfH0_max)
      
      #self.df_bin_comparision = df_bin_comparision
      #self.dfH0 = dfH0
      #self.dfH0_max = dfH0_max

      current_bin = dfH0_max.bin
      next_bin = dfH0_max.next_bin
      p_value = dfH0_max.p_value
      n_current_bin = dfH0_max.n_current_bin
      n_next_bin = dfH0_max.n_next_bin
      p_current_bin = dfH0_max.p_current_bin
      p_next_bin = dfH0_max.p_next_bin

      print('current:', current_bin, 'next_bin:', next_bin ,'p_value:', p_value, 'n_current_bin:', n_current_bin, 'n_next_bin:', n_next_bin, 'p_current_bin:', p_current_bin, 'p_next_bin:', p_next_bin)

      print(current_bin, ' now is', next_bin)
      dfDerivedData.loc[dfDerivedData['bin'] == current_bin, 'bin'] = next_bin
      dfDerivedData['bin'] = dfDerivedData['bin'].astype('int64')

      with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(dfDerivedData.groupby('bin').aggregate({'countColumn' : np.sum, targetColumn : [np.sum, np.mean], feature : [np.min, np.max]}))
    
    return dfDerivedData.sort_values('bin').reset_index(drop=True)

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class ManualDropping(BaseEstimator, TransformerMixin):

  def __ini__(self, drop_list = []):
    self.drop_list = drop_list
  
  def insert_new_drops(self, drop_list = []):
    self.drop_list += drop_list
  
  def fit(self, x, y=None):
    return
  
  def transform(self, x):
    res = x.drop([d for d in self.drop_list if d in x.columns], axis=1)
    if len([d for d in self.drop_list if d not in x.columns]) > 0:
      print('Warning: the following features weren\'t found in current DataFrame')
      print([d for d in self.drop_list if d not in x.columns])
    return res
  
  def fit_transform(self, x):
    return self.transform(x)

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

class ValueReplacement(BaseEstimator, TransformerMixin):
  
  def __ini__(self, from_to_rule_tuples = []):
    if np.max(np.array([len(t) for t in from_to_rule_tuples]) != 3):
      raise NotImplementedError('All tuples must have length equal to 3')

    for feature, from_, to_ in self.from_to_rule_tuples:
      if not type(from_) is type(to_):
        raise NotImplementedError(str(from_) + ' and ' + str(to_) + ' don\'t have same types')
        
    self.from_to_rule_tuples = from_to_rule_tuples
  
  def insert_new_rules(self, from_to_rule_tuples = []):
    if np.max(np.array([len(t) for t in from_to_rule_tuples]) != 3):
      raise NotImplementedError('All tuples must have length equal to 3')

    for feature, from_, to_ in self.from_to_rule_tuples:
      if not type(from_) is type(to_):
        raise NotImplementedError(str(from_) + ' and ' + str(to_) + ' don\'t have same types')
    
    self.from_to_rule_tuples += from_to_rule_tuples
  
  def fit(self, x, y=None):
    return

  def transform(self, x):
    res = x
    for feature, from_, to_ in self.from_to_rule_tuples:
      if feature not in self.res.columns:
        raise NotImplementedError('Referenced feature does not exists in trained DataFrame')
      res.loc[res[feature] == from_, feature] = to_
    return res

  def fit_transform(self, x):
    return self.transform(x)

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureConfiguration(BaseEstimator, TransformerMixin):

  def __init__(self, file = ''):

    self.data_feature_config = pd.read_csv(file, sep=';')

    self.feature_config = {}
    self.feature_dtypes = {}
    self.key_feature_list = []
    self.target_feature_list = []
    self.drop_feature_list = []
    self.interval_feature_list = []
    self.nominal_feature_list = []
    self.unknown_feature_list = []

    for index, row in self.data_feature_config.iterrows():
      if row['NAME'] in self.feature_config:
        raise ValueError('Duplicate feature configuration for ' + row['NAME'])
      self.feature_config[row['NAME']] = {'ROLE':row['ROLE'], 'LEVEL':row['LEVEL'], 'DROP':row['DROP']}
      if row['ROLE'] == 'TARGET':
        self.feature_dtypes[row['NAME']] = 'float64' if row['LEVEL'] in ['INTERVAL', 'BINARY'] else 'category' if row['LEVEL'] in ['NOMINAL', 'ORDINAL'] else 'object'
      else:
        self.feature_dtypes[row['NAME']] = 'float64' if row['LEVEL'] == 'INTERVAL' else 'category' if row['LEVEL'] in ['NOMINAL', 'ORDINAL', 'BINARY'] else 'object'
      if row['DROP'] == 'Y':
        self.drop_feature_list.append(row['NAME'])
      elif row['ROLE'] == 'TARGET':
        self.target_feature_list.append(row['NAME'])
      elif row['ROLE'] == 'ID':
        self.key_feature_list.append(row['NAME'])
      elif row['ROLE'] == 'INPUT' and row['LEVEL'] == 'INTERVAL':
        self.interval_feature_list.append(row['NAME'])
      elif row['ROLE'] == 'INPUT' and row['LEVEL'] in ['NOMINAL', 'ORDINAL', 'BINARY']:
        self.nominal_feature_list.append(row['NAME'])
      else:
        self.unknown_feature_list.append(row['NAME'])

  def fit(self, x, y=None):
    return

  def transform(self, x):
    res = x
    for feature, feature_dtype in self.feature_dtypes.items():
      if feature in self.res.columns:
        res[feature] = res[feature].astype(feature_dtype)
    res = res.drop(self.drop_feature_list, axis=1)
    return res

  def fit_transform(self, x):
    return self.transform(x)